***
@author: jmeier<br>
date: december 2023<br>
short description: <br>
-  reads in all entries for communites in Germany from a .xlsx file. <br>
-  parses them according to the .xlsx rules to read in Bundesländer, Regierungsbezirke, Landkreise, Verwaltungsbezirke and Gemeinden<br>
-  saves the data in a sqlite database <br>
***

In [1]:
import pandas as pd
import sqlite3
import numpy as np

In [2]:
# Reads relevant columns from file "AuszugGV3QAktuell.xlsx" (list of all communites). 
# "Bevölkerung" was read as string, because " " was used as thousands delimiter and read in did not work well
df = pd.read_excel("AuszugGV3QAktuell.xlsx", sheet_name=1, skiprows=[0,1,2,3,4], skipfooter=12, usecols="A:P", decimal=",", thousands=" ",
                   names=["Satzart", "Textkennzeichen", "Land",	"RB",	"Kreis",	"VB",	"Gem",	"Gemeindename",	"Fläche km2",
                          	"Bevölkerung-insgesamt",	"Bevölkerung-männlich",	"Bevölkerung-weiblich",	"Bevölkerung-je km2",
                            "Postleitzahl",	"Längengrad",	"Breitengrad"],
                            dtype={"Satzart": np.str_, "Textkennzeichen": np.str_, "Land": np.str_, "RB": np.str_, "Kreis": np.str_, "VB": np.str_, "Gem": np.str_, 
                                   #"Fläche km2": np.float32,
                                   "Bevölkerung-insgesamt": np.str_, "Bevölkerung-männlich": np.str_,  "Bevölkerung-weiblich": np.str_,  "Bevölkerung-je km2": np.str_,
                                   "Postleitzahl": np.str_#, "Längengrad": np.float32, "Breitengrad": np.float32 
                                   }
                            )
                            #"Reisegebiete-Schlüssel",	"Reisegebiete-Bezeichnung",	"Grad der Verstädterung-Schlüssel",	"Grad der Verstädterung-Bezeichnung"])
df.head(15)

,Satzart,Textkennzeichen,Land,RB,Kreis,VB,Gem,Gemeindename,Fläche km2,Bevölkerung-insgesamt,Bevölkerung-männlich,Bevölkerung-weiblich,Bevölkerung-je km2,Postleitzahl,Längengrad,Breitengrad
0,10,NaN,01,NaN,NaN,NaN,NaN,Schleswig-Holstein,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40,41,01,0,01,NaN,NaN,"Flensburg, Stadt",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,50,50,01,0,01,0000,NaN,"Flensburg, Stadt",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,60,61,01,0,01,0000,000,"Flensburg, Stadt",56.73,92550,45874,46676,1631,24937,9.437510,54.782520
4,40,41,01,0,02,NaN,NaN,"Kiel, Landeshauptstadt",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,50,50,01,0,02,0000,NaN,"Kiel, Landeshauptstadt",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,60,61,01,0,02,0000,000,"Kiel, Landeshauptstadt",118.65,247717,120149,127568,2088,24103,10.137270,54.321775
7,40,41,01,0,03,NaN,NaN,"Lübeck, Hansestadt",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,50,50,01,0,03,0000,NaN,"Lübeck, Hansestadt",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,60,61,01,0,03,0000,000,"Lübeck, Hansestadt",214.19,218095,104723,113372,1018,23552,10.683932,53.866269


In [3]:
conn = sqlite3.connect("communities.sqlite")
cur = conn.cursor()

#### Next few blocks generate the tables (Bundesländer, Regierungsbezirke, Landkreise, Verwaltungsbezirke, Gemeinden) of the database

In [4]:
#cur.execute("DROP TABLE IF EXISTS Land")

cur.execute("""CREATE TABLE IF NOT EXISTS Land (
                id  INTEGER PRIMARY KEY,
                land_nr TEXT,
                name TEXT UNIQUE
            )""")

In [5]:
#cur.execute("DROP TABLE IF EXISTS RB")

cur.execute("""CREATE TABLE IF NOT EXISTS RB (
                id  INTEGER PRIMARY KEY,
                rb_nr TEXT,
                name TEXT UNIQUE,
                land_id INTEGER
            )""")

In [6]:
#cur.execute("DROP TABLE IF EXISTS Kreis")

cur.execute("""CREATE TABLE IF NOT EXISTS Kreis (
                id  INTEGER PRIMARY KEY,
                kreis_nr TEXT,
                name TEXT,
                rb_id INTEGER,
                textkennzeichen INTEGER,
                UNIQUE (kreis_nr, name)
            )""")

In [7]:
#cur.execute("DROP TABLE IF EXISTS VB")

cur.execute("""CREATE TABLE IF NOT EXISTS VB (
                id  INTEGER PRIMARY KEY,
                vb_nr TEXT,
                name TEXT,
                kreis_id INTEGER,
                textkennzeichen INTEGER,
                UNIQUE (vb_nr, kreis_id, name)
            )""")

In [8]:
# Gemeinde Table contains man columns to save the travel distances and times (from different sources r5py, ors or google)

#cur.execute("DROP TABLE IF EXISTS Gemeinde")

cur.execute("""CREATE TABLE IF NOT EXISTS Gemeinde (
                id  INTEGER PRIMARY KEY,
                gem_nr TEXT,
                name TEXT,
                vb_id INTEGER,
                textkennzeichen INTEGER,
                fläche REAL,
                bevölkerung_insgesamt INTEGER,	
                bevölkerung_männlich INTEGER,
            	bevölkerung_weiblich INTEGER,
                bevölkerung_je_km2 INTEGER,
                postleitzahl TEXT,
                längengrad REAL,
                breitengrad REAL,
                r5py_lmu INTEGER,
                r5py_tum INTEGER,
                r5py_würzburg INTEGER,
                r5py_erlangen INTEGER,
                r5py_regensburg INTEGER,               
                r5py_augsburg INTEGER,
                r5py_retrieved INTEGER,
                ors_distance_lmu INTEGER,
                ors_time_lmu INTEGER,
                ors_distance_tum INTEGER,
                ors_time_tum INTEGER,
                ors_distance_würzburg INTEGER,
                ors_time_würzburg INTEGER,
                ors_distance_erlangen INTEGER,
                ors_time_erlangen INTEGER,
                ors_distance_regensburg INTEGER,
                ors_time_regensburg INTEGER,
                ors_distance_augsburg INTEGER,
                ors_time_augsburg INTEGER,
                ors_retrieved INTEGER, 
                google_thur7am_car_distance_lmu INTEGER,
                google_thur7am_car_time_lmu INTEGER,
                google_thur7am_car_distance_tum INTEGER,
                google_thur7am_car_time_tum INTEGER,
                google_thur7am_car_distance_würzburg INTEGER,
                google_thur7am_car_time_würzburg INTEGER,
                google_thur7am_car_distance_erlangen INTEGER,
                google_thur7am_car_time_erlangen INTEGER,
                google_thur7am_car_distance_regensburg INTEGER,
                google_thur7am_car_time_regensburg INTEGER,
                google_thur7am_car_distance_augsburg INTEGER,
                google_thur7am_car_time_augsburg INTEGER,
                google_thur7am_transit_distance_lmu INTEGER,
                google_thur7am_transit_time_lmu INTEGER,
                google_thur7am_transit_distance_tum INTEGER,
                google_thur7am_transit_time_tum INTEGER,
                google_thur7am_transit_distance_würzburg INTEGER,
                google_thur7am_transit_time_würzburg INTEGER,
                google_thur7am_transit_distance_erlangen INTEGER,
                google_thur7am_transit_time_erlangen INTEGER,
                google_thur7am_transit_distance_regensburg INTEGER,
                google_thur7am_transit_time_regensburg INTEGER,
                google_thur7am_transit_distance_augsburg INTEGER,
                google_thur7am_transit_time_augsburg INTEGER,
                google_retrieved INTEGER,               
                UNIQUE (gem_nr, vb_id, name)

            )""")

In [9]:
print(len(df))

16035


In [10]:
# parses the .xlsx entries and saves them in the respective database table

for i in range(len(df)):

    if df.iloc[i]["Satzart"] == "10":
        cur.execute("INSERT OR IGNORE INTO LAND (land_nr, name) VALUES (?, ?)", (df.iloc[i]["Land"], df.iloc[i]["Gemeindename"]))
        cur.execute("SELECT id FROM Land WHERE name=?", (df.iloc[i]["Gemeindename"], ))
        land_id = cur.fetchone()[0]
        #land_id = cur.lastrowid

        cur.execute("INSERT OR IGNORE INTO RB (rb_nr, name, land_id) VALUES ('0', ?, ?)", (df.iloc[i]["Gemeindename"], land_id))
        cur.execute("SELECT id FROM RB WHERE name=?", (df.iloc[i]["Gemeindename"], ))
        rb_id = cur.fetchone()[0]
        #rb_id = cur.lastrowid
        #print(df.iloc[i])

    elif df.iloc[i]["Satzart"] == "20":
        cur.execute("INSERT OR IGNORE INTO RB (rb_nr, name, land_id) VALUES (?, ?, ?)", (df.iloc[i]["RB"], df.iloc[i]["Gemeindename"], land_id))
        cur.execute("SELECT id FROM RB WHERE name=?", (df.iloc[i]["Gemeindename"], ))
        rb_id = cur.fetchone()[0]
        #rb_id = cur.lastrowid

    elif df.iloc[i]["Satzart"] == "40":
        cur.execute("INSERT OR IGNORE INTO Kreis (kreis_nr, name, rb_id, textkennzeichen) VALUES (?, ?, ?, ?)", (df.iloc[i]["Kreis"], df.iloc[i]["Gemeindename"], rb_id, int(df.iloc[i]["Textkennzeichen"])))
        cur.execute("SELECT id FROM Kreis WHERE name=? AND kreis_nr=?", (df.iloc[i]["Gemeindename"], df.iloc[i]["Kreis"]))
        kreis_id = cur.fetchone()[0]
        #kreis_id = cur.lastrowid

    elif df.iloc[i]["Satzart"] == "50":
        cur.execute("INSERT OR IGNORE INTO VB (vb_nr, name, kreis_id, textkennzeichen) VALUES (?, ?, ?, ?)", (df.iloc[i]["VB"], df.iloc[i]["Gemeindename"], kreis_id, int(df.iloc[i]["Textkennzeichen"])))
        cur.execute("SELECT id FROM VB WHERE name=? AND vb_nr=? AND kreis_id =?", (df.iloc[i]["Gemeindename"], df.iloc[i]["VB"], kreis_id))
        vb_id = cur.fetchone()[0]
        #vb_id = cur.lastrowid

    elif df.iloc[i]["Satzart"] == "60":
        cur.execute("""INSERT OR IGNORE INTO    Gemeinde    (gem_nr,            name,                       vb_id,  textkennzeichen,                    fläche,                     bevölkerung_insgesamt,                      bevölkerung_männlich,                       bevölkerung_weiblich,                       bevölkerung_je_km2,                     Postleitzahl,               Längengrad,                 Breitengrad,                r5py_retrieved, ors_retrieved,  google_retrieved) 
                                                VALUES      (?,                 ?,                          ?,      ?,                                  ?,                          ?,                                          ?,                                          ?,                                          ?,                                      ?,                          ?,                          ?,                          0,              0,              0)""", 
                                                            (df.iloc[i]["Gem"], df.iloc[i]["Gemeindename"], vb_id,  int(df.iloc[i]["Textkennzeichen"]), df.iloc[i]["Fläche km2"],   int(df.iloc[i]["Bevölkerung-insgesamt"]),   int(df.iloc[i]["Bevölkerung-männlich"]),    int(df.iloc[i]["Bevölkerung-weiblich"]),    int(df.iloc[i]["Bevölkerung-je km2"]),  df.iloc[i]["Postleitzahl"], df.iloc[i]["Längengrad"],   df.iloc[i]["Breitengrad"]))
        #cur.execute("SELECT id FROM Gemeinde WHERE name=? AND gem_nr=? AND vb_id=?", (df.iloc[i]["Gemeindename"], df.iloc[i]["Gem"], vb_id))
              
        #Kreis_id = cur.lastrowid


    #print(df.iloc[i])

    #if i >10:
    #    break

In [11]:
conn.commit()
cur.close()
conn.close()